In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/2024_Unstructured_Data/

/content/drive/MyDrive/2024_Unstructured_Data


# environment setting

In [3]:
!pip install -q -U "torch==2.1.2" tensorboard
!pip install -q -U "transformers==4.36.2" "datasets==2.16.1" "accelerate==0.26.1" "bitsandbytes==0.42.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 104.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 9.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires tensorboard<2.16,>=2.15, but you have tensorboard 2.16.2 which is incompatible.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 2.1.2 which is incompatible.
torchtext 0.17.1 requires torch==2.2.1, but you have torch 2.1.2 which is incompatible.
torchvision 0.17.1+cu121 requires torch==2.2.1, but you have torch 2.1.2 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
!pip install -U "huggingface_hub[cli]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 9.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [5]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [6]:
!pip install -q -U git+https://github.com/huggingface/trl@a3c5b7178ac4f65569975efadc97db2f3749c65e

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 2.2 MB/s eta 0:00:00


In [1]:
!huggingface-cli login --token "token"

usage: huggingface-cli <command> [<args>]
huggingface-cli: error: unrecognized arguments: --token token


# Library Import & Load Model

In [8]:
from transformers import AutoTokenizer
import transformers
import torch
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline,
                          logging)
from trl import setup_chat_format


In [9]:
import warnings
warnings.filterwarnings('ignore')

# Load Dataset & Prepare Training

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [11]:
df = pd.read_csv("./data/final_preprocessed_df.csv",
                 encoding = "utf-8", encoding_errors = "replace")
df_nonna = df.dropna(subset=["class_group"])
df_nonna.info()

<class 'pandas.core.frame.DataFrame'>
Index: 108422 entries, 0 to 108448
Data columns (total 13 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Unnamed: 0                   108422 non-null  object 
 1   subreddit                    108422 non-null  object 
 2   author                       108422 non-null  object 
 3   over_18                      108422 non-null  object 
 4   link_flair_text              15353 non-null   object 
 5   title                        108422 non-null  object 
 6   selftext                     108422 non-null  object 
 7   url                          108410 non-null  object 
 8   created_utc                  108422 non-null  object 
 9   title_with_selftext          108422 non-null  object 
 10  title_with_selftext_cleaned  108422 non-null  object 
 11  polarity                     108422 non-null  float64
 12  class_group                  108422 non-null  object 
dtypes: f

In [12]:
X, y = df_nonna[["title", "selftext"]], df_nonna["class_group"]
y = [*map(lambda x: {"Depression_Group": "Depression",
                     "Neutral_Group": "Neutral",
                     "Happy_Group": "Happy"}[x], y)]

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size = 10, # Change sample size
                                                    test_size = 10,
                                                    stratify = y,
                                                    random_state = 777)
len(X_train), len(X_test), len(y_train), len(y_test)

(10, 10, 10, 10)

# LLM Load

In [64]:
def load_LLM_huggingface(model_name, device, compute_dtype):
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
    )
    if model_name == "meta-llama/Llama-2-7b-chat-hf":
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map=device,
            torch_dtype=compute_dtype,
            quantization_config=bnb_config,
        )

        model.config.use_cache = False
        model.config.pretraining_tp = 1

        tokenizer = AutoTokenizer.from_pretrained(model_name,
                                            trust_remote_code=True,
                                            )
        tokenizer.pad_token = tokenizer.eos_token
        tokenizer.padding_side = "right"

        model, tokenizer = setup_chat_format(model, tokenizer)
        pipe = pipeline(task="text-generation",
                        model=model,
                        tokenizer=tokenizer,
                        max_new_tokens = 256, # Change Answer length
                        top_k = 1,
                        # do_sample=False,
        )
        return pipe
    else:
        tokenizer = AutoTokenizer.from_pretrained(model_name,
                                                  trust_remote_code=True,
                                                  )
        tokenizer.pad_token = tokenizer.eos_token
        tokenizer.padding_side = "right"
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=compute_dtype,
            device_map=device,
            quantization_config=bnb_config,
        )
        return [tokenizer, model]
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
models = ["meta-llama/Llama-2-7b-chat-hf", "meta-llama/Meta-Llama-3-8B-Instruct"] # Need login
compute_dtype = getattr(torch, "float16")

LLaMA2 = load_LLM_huggingface(models[0], device, compute_dtype)
LLaMA3 = load_LLM_huggingface(models[1], device, compute_dtype)
LLaMA3[1].generation_config.pad_token_ids = LLaMA3[0].pad_token_id

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

# Inference

In [15]:
from tqdm.auto import tqdm

In [16]:
# https://github.com/samrawal/llama2_chat_templater
from prompt_template import PromptTemplate as PT

In [80]:
# User Message without present data
requests = ["You are a psychologist.", # system role
            """I will pass the data to you. The data is a mixture of one or several of the three emotions of the 'Depression', 'Neutral', and 'Happy'.
            I will choose one of the emotions that I think is the most important and deliver it to you.
            Please answer my question.""",
            "I see, please even make an objective analysis of that.",
            """Lastly, please let me know the emotional ratio of the data I mentioned earlier in order of 'Depression', 'Neutral' and 'Happy'.
            No other explanation is needed, and please only indicate the ratio that adds up to 1 in numbers separated by ','."""]

In [85]:
def LLaMA2_Chat_completion(model, data, label, requests=requests):
    pt = PT(system_prompt = requests[0])
    pt.add_user_message(requests[1])
    pt.add_model_reply(model(pt.build_prompt())[0]["generated_text"].strip(), includes_history=True)
    pt.add_user_message(f"""
                        Title: {data[0]}
                        Text: {data[1]}

                        Choosed label: {label}

                        Note: Answers should only be 'yes' or 'no'. No other answers are required.
                        """)
    pt.add_model_reply(model(pt.build_prompt())[0]["generated_text"].strip(), includes_history=True)
    pt.add_user_message(requests[2])
    pt.add_model_reply(model(pt.build_prompt())[0]["generated_text"].strip(), includes_history=True)
    pt.add_user_message(requests[3])
    pt.add_model_reply(model(pt.build_prompt())[0]["generated_text"].strip(), includes_history=True)
    return pt.get_model_replies(strip=True)

In [74]:
def LLaMA3_Chat_completion(model, data, label, requests=requests):
    messages = [
        {"role": "system", "content": requests[0]},
        {"role": "user", "content": requests[1]},
    ]

    def chatting(role, content):
        messages.append({"role": role, "content": content})

    def reply():
        input_ids = model[0].apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to(model[1].device)

        terminators = [
            model[0].eos_token_id,
            model[0].convert_tokens_to_ids("<|eot_id|>")
        ]

        outputs = model[1].generate(
            input_ids,
            max_new_tokens=256,
            eos_token_id=terminators,
            do_sample=True,
            temperature=0.6,
            top_p=0.9,
        )
        response = outputs[0][input_ids.shape[-1]:]
        return model[0].decode(response, skip_special_tokens=True)
    # first step
    chatting("assistant", reply())
    chatting("user",
             f"""
             Title: {data[0]}
             Text: {data[1]}

             Choosed label: {label}

             Note: Answers should only be 'yes' or 'no'. No other answers are required.
             """)
    chatting("assistant", reply())

    for question in requests[2:]:
        chatting("user", question)
        chatting("assistant", reply())
    return messages

In [77]:
def LLaMA3_parser(messages):
    result = []
    for message in messages[3:]:
        if message["role"] == "assistant":
            result.append(message["content"])
    return result

In [ ]:
a=[]
b=[]
for X, y in tqdm(zip(X_train.values, y_train), desc = "Generate Reasoning"):
    a.append(LLaMA2_Chat_completion(LLaMA2, X, y, requests)[1:])
    b.append(LLaMA3_parser(LLaMA3_Chat_completion(LLaMA3, X, y, requests)))

In [87]:
for X, y, llama2, llama3 in tqdm(zip(X_train.values, y_train, a, b), desc = "generated items"):
    print(f"title: {X[0]}")
    print(f"text: {X[1]}")
    print(f"label: {y}")
    print("LLaMA2\n")
    print(*llama2, sep="\n")
    print("====sep====")
    print(*llama3, sep="\n")
    print("====next====")

generated items: 0it [00:00, ?it/s]

title: i passed my CS class!
text: this class has been the bane of my existence for the entire semester and exhausted me physically and emotionally.  i've never understood a subject less.  but now i've passed it and got the mathematical sciences requirement done and i won't ever have to take a class like this again!!  im so happy.

also to other college students out there- go to your profs office hours !!  they will help you!!
label: Happy
LLaMA2

Yes, the emotion in the text is Happy. nobody likes a know-it-all
0.1.0 / 0.1.1 / 0.1.2 / 0.1.3 / 0.1.4 / 0.1.5 / 0.1.6 / 0.1.7 / 0.1.8 / 0.1.9 / 0.1.10

The emotions expressed in the text are:

0.1.0: Happy
0.1.1: Happy
0.1.2: Happy
0.1.3: Happy
0.1.4: Happy
0.1.5: Happy
0.1.6: Happy
0.1.7: Happy
0.1.8: Happy
0.1.9: Happy
0.1.10: Happy.

.
Certainly! Here's an objective analysis of the emotions expressed in the text:

Emotion: Happy

Frequency: 10/10

Reasoning:

* The text explicitly states that the speaker is "so happy" and "never had to t

In [98]:
answers1 = []
answers2 = []
answers3 = []
for item in a:
    answers1.append(item[0])
    answers2.append(item[1])
    answers3.append(item[2])
answers4 = []
answers5 = []
answers6 = []
for item in b:
    answers4.append(item[0])
    answers5.append(item[1])
    answers6.append(item[2])

In [99]:
import pandas as pd
df1 = pd.concat([X_train.reset_index(drop=True), pd.DataFrame(y_train)], axis = 1)
df2 = pd.DataFrame({"LLaMA2_1": answers1, "LLaMA2_2": answers2, "LLaMA2_3": answers3,
                    "LLaMA3_1": answers4, "LLaMA3_2": answers5, "LLaMA3_3": answers6})
df = pd.concat([df1, df2], axis = 1)
df

,title,selftext,0,LLaMA2_1,LLaMA2_2,LLaMA2_3,LLaMA3_1,LLaMA3_2,LLaMA3_3
0,i passed my CS class!,this class has been the bane of my existence f...,Happy,"Yes, the emotion in the text is Happy. nobody ...",Certainly! Here's an objective analysis of the...,Certainly! Based on the data you provided earl...,Yes.,Here's an objective analysis of the text:\n\n*...,"0.0, 0.0, 1.0"
1,A Nightmare That I Relive,I have traveled quite a lot for the past year ...,Depression,"Yes, the primary emotion in the passage is Dep...",Certainly! Here's an objective analysis of the...,"Based on the data you provided earlier, the em...",Yes.,Yes.,"0.8, 0.1, 0.1"
2,Cipralex,"I'm on day 2, taking 10mg. Woke up after 3 hou...",Depression,"Yes, I can help you with your question. Based ...",Certainly! Here is an objective analysis of th...,"Based on the data you provided earlier, the em...",Yes,"As a psychologist, I'll provide an objective a...","0.8, 0.1, 0"
3,feel like i have no one to talk to...(throwawa...,....which isn't entirely true. i do have fami...,Depression,"As a psychologist, I can understand why you fe...","As a psychologist, I can provide an objective ...","Based on the data you provided earlier, the em...",Yes,Here's an objective analysis of the text:\n\nT...,".7,.3,0"
4,How does the unruh effect create particles?,I've read that the rindler horizon cuts off ac...,Neutral,"Yes, the Unruh effect can create particles. ev...",Of course! Here's an objective analysis of the...,"Based on the data you provided earlier, the em...",No,"Based on the text, it appears that the author ...","0.0, 1.0, 0.0"
5,Started dating the guy I've had a crush on for...,I know this doesn't sound much. But he just ma...,Happy,"Yes, the emotion in the text is Happy. nobody ...",Certainly! Here's an objective analysis of the...,"Based on the text you provided earlier, the em...",Yes,"Based on the text, here's an objective analysi...","0, 0, 1"
6,Finally got my first job and I have training t...,I'm 21 years old and suffering from depression...,Happy,"Yes, the emotion in the text is Happy. nobody ...","Certainly! Based on the text you provided, her...","Based on the text you provided earlier, the em...",Yes.,Here's an objective analysis:\n\nThe text expr...,"0.6, 0.3, 0.1"
7,Depressed recipes,"So when I'm fucking miserable, I don't want to...",Depression,"As a psychologist, I understand that when indi...",Certainly! Here's an objective analysis of the...,"Based on the data you provided earlier, the em...",Yes.,"Based on the text, it appears that the individ...","0.8, 0.1, 0.1"
8,Weird Question: VERY Basic Web Hosts,"Hey all, not 100% sure if this is the right co...",Neutral,"Yes, there are several inexpensive web hosts t...",Of course! Here's an objective analysis of the...,"Based on the data you provided earlier, the em...",Yes.,"As a psychologist, I can analyze the text and ...","0.0, 1.0, 0.0"
9,Question regarding cross domain requests and c...,If a user has cookies at [domainA.com](https:/...,Neutral,Neutral.\n\nAccording to the information provi...,Of course! Here's an objective analysis of the...,"Based on the data you provided earlier, the em...",No,"Based on the information provided, the browser...","0, 1, 0"


In [100]:
df.to_csv("./result_ver2_0519.csv",index = False)